In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [82.3 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-10 00:38:50--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.67MB/s    in 0.2s    

2022-04-10 00:38:50 (4.67 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "vine", "total_votes", "verified_purchase"]).drop_duplicates(['review_id'])
vine_df.show()

+--------------+-----------+-------------+----+-----------+-----------------+
|     review_id|star_rating|helpful_votes|vine|total_votes|verified_purchase|
+--------------+-----------+-------------+----+-----------+-----------------+
|R101CJ0JBI9J43|          5|            6|   N|          6|                Y|
|R101P7GUI5O1WG|          5|            0|   N|          0|                Y|
|R102LL91IX39PK|          5|            0|   N|          0|                Y|
|R102YVSWWEAP5K|          5|            0|   N|          0|                Y|
|R1042WRU1WDUHY|          5|            7|   N|          8|                Y|
|R104G7VC1XPVOO|          5|            1|   N|          1|                Y|
|R10557FTU9GHY8|          5|            0|   N|          0|                Y|
|R105B7Y5ALJ45D|          5|            1|   N|          1|                Y|
|R105C41QCCFLIN|          5|            0|   N|          0|                Y|
|R105MO44HU33PJ|          5|            0|   N|          0|     

In [7]:
# create df with total_votes greaterthan 20
total_votes_df = vine_df.filter("total_votes>20")
total_votes_df.show()

+--------------+-----------+-------------+----+-----------+-----------------+
|     review_id|star_rating|helpful_votes|vine|total_votes|verified_purchase|
+--------------+-----------+-------------+----+-----------+-----------------+
| R11YMYHSPNGA9|          5|          112|   N|        117|                Y|
|R12E1DPQO0PSC5|          5|           55|   N|         57|                N|
|R12Q28Q3H61LJ4|          5|           47|   N|         47|                Y|
|R12ZZBLV20KHZN|          5|           21|   N|         24|                Y|
|R14RP0JIFZOU0F|          1|            9|   N|         26|                Y|
|R15ERI6B6F5POS|          5|           27|   N|         29|                Y|
|R162U5PSTTFDRY|          5|           30|   N|         31|                N|
|R17HY9US9BLH1T|          1|           75|   N|         81|                Y|
|R18E8DQJPSW932|          4|           25|   N|         26|                N|
|R18YQ6WBZM7CTR|          1|           25|   N|         25|     

In [8]:
# filter df to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
new_col_df = total_votes_df.withColumn("Div", total_votes_df["helpful_votes"]/total_votes_df["total_votes"])
new_df = new_col_df.filter("Div>=0.5")
newer_df=new_df.drop("Div")
newer_df.show()

+--------------+-----------+-------------+----+-----------+-----------------+
|     review_id|star_rating|helpful_votes|vine|total_votes|verified_purchase|
+--------------+-----------+-------------+----+-----------+-----------------+
| R11YMYHSPNGA9|          5|          112|   N|        117|                Y|
|R12E1DPQO0PSC5|          5|           55|   N|         57|                N|
|R12Q28Q3H61LJ4|          5|           47|   N|         47|                Y|
|R12ZZBLV20KHZN|          5|           21|   N|         24|                Y|
|R15ERI6B6F5POS|          5|           27|   N|         29|                Y|
|R162U5PSTTFDRY|          5|           30|   N|         31|                N|
|R17HY9US9BLH1T|          1|           75|   N|         81|                Y|
|R18E8DQJPSW932|          4|           25|   N|         26|                N|
|R18YQ6WBZM7CTR|          1|           25|   N|         25|                Y|
|R19V1JU0BUV5K4|          5|           46|   N|         48|     

In [9]:
# filter vine reviews
from pyspark.sql.functions import col
vine_yes_df = newer_df.filter(col("vine")=='Y')
vine_yes_df.show()

+--------------+-----------+-------------+----+-----------+-----------------+
|     review_id|star_rating|helpful_votes|vine|total_votes|verified_purchase|
+--------------+-----------+-------------+----+-----------+-----------------+
|R2XLAQL0E2UPWQ|          4|           33|   Y|         34|                N|
|R3F6KXGIFXQ3B6|          4|           40|   Y|         43|                N|
| R761OO343LFQW|          3|           82|   Y|         93|                N|
|R1XZXTP2UJR37K|          4|          127|   Y|        128|                N|
|R1241UCQGRHCNH|          5|           72|   Y|         76|                N|
|R1CN8C3E6GNUFO|          1|           29|   Y|         33|                N|
|R2D1YV9RG1BD7K|          4|           18|   Y|         22|                N|
|R380KQVSEH3PQT|          5|           26|   Y|         28|                N|
| RAZ7HBRW7G0AX|          5|          313|   Y|        333|                N|
|R21MHBZMJWQL1K|          4|           34|   Y|         36|     

In [10]:
vine_no_df = newer_df.filter(col("vine")=='N')
vine_no_df.show()

+--------------+-----------+-------------+----+-----------+-----------------+
|     review_id|star_rating|helpful_votes|vine|total_votes|verified_purchase|
+--------------+-----------+-------------+----+-----------+-----------------+
| R11YMYHSPNGA9|          5|          112|   N|        117|                Y|
|R12E1DPQO0PSC5|          5|           55|   N|         57|                N|
|R12Q28Q3H61LJ4|          5|           47|   N|         47|                Y|
|R12ZZBLV20KHZN|          5|           21|   N|         24|                Y|
|R15ERI6B6F5POS|          5|           27|   N|         29|                Y|
|R162U5PSTTFDRY|          5|           30|   N|         31|                N|
|R17HY9US9BLH1T|          1|           75|   N|         81|                Y|
|R18E8DQJPSW932|          4|           25|   N|         26|                N|
|R18YQ6WBZM7CTR|          1|           25|   N|         25|                Y|
|R19V1JU0BUV5K4|          5|           46|   N|         48|     

In [17]:
# calculate total # of reviews, # of 5 star reviews and % of 5 star reviews for YES reviews
total_reviews = vine_yes_df.agg({"review_id": "count"})
total_reviews.show()
five_starsYes = vine_yes_df.filter("star_rating=5").agg({"star_rating": "count"})
five_starsYes.show()



+----------------+
|count(review_id)|
+----------------+
|             162|
+----------------+

+------------------+
|count(star_rating)|
+------------------+
|                63|
+------------------+



Column<b'((count(star_rating) / count(review_id)) * 100)'>

In [20]:
Yes_percent = five_starsYes.collect()[0][0]/total_reviews.collect()[0][0]*100
Yes_percent

38.88888888888889

In [21]:
# calculate total # of reviews, # of 5 star reviews and % of 5 star reviews for NO reviews
total_reviews = vine_no_df.agg({"review_id": "count"})
total_reviews.show()
five_starsNo = vine_no_df.filter("star_rating=5").agg({"star_rating": "count"})
five_starsNo.show()
No_percent = five_starsNo.collect()[0][0]/total_reviews.collect()[0][0]*100
No_percent

+----------------+
|count(review_id)|
+----------------+
|           35606|
+----------------+

+------------------+
|count(star_rating)|
+------------------+
|             19444|
+------------------+



54.60877380216817